## Search for dividend growth companies TIKR data source

### Update Jan 2024

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import datetime
import pytz
import jinja2 as j2


In [3]:
j2_env = j2.Environment()
def parse_date(x):
    if x != "LTM":
        date = datetime.datetime.strptime(x, "%m/%d/%y")
        return str(date.year) + "-" + str(date.month)
    return x


def replacetonumbeR(s):
    if type(s).__name__ == "str":
        s = s.strip()
        if s == "-":
            s = 0
        else:
            s = s.replace(",","")
            if s.find("(") >= 0 and s.find(")") >= 0:
                s = s.replace("(","-").replace(")","")
            if s.find("%") >= 0:
                s = s.replace("%", "")
                s = float(s) / 100
    return s


def parse_table(t):
    t = t.dropna(how='all')
    t = t.set_index(t.columns[0])
    t = t.dropna(how='all', axis=1)
    t = t.drop([c for c in t.index if "YoY" in c])
    t = t.rename(columns=parse_date)
    t = t.applymap(lambda x:replacetonumbeR(x))
    t = t.astype(float)
    t = t.fillna(0)
    return t


def get_growth_per_year(series, year):
    x = ((series.shift(-year) / series) ** (1 / year)) - 1
    x = x.dropna()
    return x.iloc[-1] if len(x) > 0 else np.NaN


def get_series_stats(series, years=5, dispersion_metrics=True):
    metrics = { 
        "series": list(series),
        "yy_growth": [get_growth_per_year(series, i) for i in range(years, 0, -1)],
        "yy_growth_5": get_growth_per_year(series, 5),
        "yy_growth_3": get_growth_per_year(series, 3),
        "yy_growth_1": get_growth_per_year(series, 1),
        "growth_tot": series.pct_change(periods=years).iloc[-1],
        "ltm": series["LTM"],
    }
    if dispersion_metrics:
        metrics.update({
        "mean": series.mean(),
        "std": series.std(),
        "mean_z2": series[(np.abs(stats.zscore(series)) <= 2)].mean(),
        "std_z2": series[(np.abs(stats.zscore(series)) <= 2)].std()
    })
    return metrics


def get_income_stats(income: pd.DataFrame, years=5):
    result = {
        "revenues": get_series_stats(income.loc["Revenues"], dispersion_metrics=False, years=years),
        "gross": get_series_stats(income.loc["Gross Profit"], dispersion_metrics=False, years=years),
        "gross_margin": get_series_stats(income.loc["Gross Profit"] / income.loc["Revenues"], years=years),
        "gross_sga_margin": get_series_stats(-income.loc["Selling General & Admin Expenses"] / income.loc["Gross Profit"], years=years),
        "gross_depreciation_margin": get_series_stats(-income.loc["Depreciation & Amortization"] / income.loc["Gross Profit"], years=years) if "Depreciation & Amortization" in income.index else np.NaN,
        "gross_r&d_margin": get_series_stats(-income.loc["R&D Expenses"] / income.loc["Gross Profit"], years=years) if "R&D Expenses" in income.index else np.NaN,
        "operating_income": get_series_stats(income.loc["Operating Income"], years=years, dispersion_metrics=False),
        "operating_margin": get_series_stats(income.loc["Operating Income"] / income.loc["Revenues"], years=years),
        "interest_expense_margin": get_series_stats(-income.loc["Interest Expense"] / income.loc["Operating Income"], years=years),
        "net_income": get_series_stats(income.loc["Net Income to Common Excl. Extra Items"], dispersion_metrics=False, years=years),
        "net_income_margin": get_series_stats(income.loc["Net Income to Common Excl. Extra Items"] / income.loc["Revenues"], years=years),
        "diluted_shares": get_series_stats(income.loc["Weighted Average Diluted Shares Outstanding"], years=years, dispersion_metrics=False),
        "eps": get_series_stats(income.loc["Diluted EPS Excl Extra Items"], dispersion_metrics=False, years=years),
        "dividends": get_series_stats(income.loc["Dividends Per Share"], years=years),
        "payout": get_series_stats(income.loc["Dividends Per Share"] * income.loc["Weighted Average Diluted Shares Outstanding"] / income.loc["Net Income"], years=years),
        "missing_dividend_years": get_series_stats(pd.Series(((income.loc["Dividends Per Share"] == 0)|(income.loc["Dividends Per Share"].isna())).sum(), index=income.columns)),
    }

    return result
    # return { ("%s_%s" % (k, vk)): result[k][vk] for k in result.keys() for vk in result[k].keys() }


def get_balance_stats(income: pd.DataFrame, balance: pd.DataFrame, years=5):
    result = {
        "cash": get_series_stats(balance.loc["Cash And Equivalents"], years=years),
        "inventory_margin": get_series_stats(balance.loc["Inventory"] / income.loc["Revenues"], years=years),
        "accounts_recv_margin": get_series_stats(balance.loc["Accounts Receivable"] / income.loc["Revenues"], years=years), # TIKR provides NET Accounts Receivables under this name
        "current_ratio": get_series_stats(balance.loc["Total Current Assets"] / balance.loc["Total Current Liabilities"], years=years),
        "goodwill": get_series_stats(balance.loc["Goodwill"], years=years),
        "assets": get_series_stats(balance.loc["Total Assets"], years=years),
        "roa": get_series_stats(income.loc["Net Income to Common Excl. Extra Items"] / balance.loc["Total Assets"], years=years),
        "net_debt": get_series_stats(balance.loc["Net Debt"], years=years),
        "debt_to_earnings": get_series_stats(balance.loc["Net Debt"] / income.loc["Net Income"], years=years),
        "debt_to_equity": get_series_stats(balance.loc["Net Debt"] / balance.loc["Total Equity"], years=years),
        "pref_shares": get_series_stats(balance.loc["Total Preferred Equity"], years=years) if "Total Preferred Equity" in balance.index else np.NaN,
        "retained_earnings": get_series_stats(balance.loc["Retained Earnings"], years=years, dispersion_metrics=False),
        "treasury_stock": get_series_stats(balance.loc["Treasury Stock"], years=years) if "Treasury Stock" in balance.index else np.NaN,
        "roe": get_series_stats(income.loc["Net Income to Common Excl. Extra Items"] / balance.loc["Total Equity"], years=years),
    }

    return result


def get_cash_stats(income: pd.DataFrame, cash: pd.DataFrame, years=5):
    fcfe = cash.loc["Free Cash Flow"] + (cash.loc["Total Debt Issued"] + cash.loc["Total Debt Repaid"])
    result = {
        "operating_cashflow": get_series_stats(cash.loc["Cash from Operations"], years=years),
        "capital_intensity": get_series_stats(-cash.loc["Capital Expenditure"] / income.loc["Net Income"], years=years),
        "fcf": get_series_stats(cash.loc["Free Cash Flow"], years=years),
        "fcf_margins": get_series_stats(cash.loc["Free Cash Flow"] / income.loc["Revenues"], years=years),
        "earnings_to_fcf": get_series_stats(income.loc["Net Income"] / cash.loc["Free Cash Flow"], years=years),
        "dividends_to_fcfe": get_series_stats(-cash.loc["Common Dividends Paid"] / fcfe, years=years),
        "buybacks_to_fcfe": get_series_stats(-cash.loc["Repurchase of Common Stock"] / fcfe, years=years),
        "d&b_to_fcfe": get_series_stats((-cash.loc["Repurchase of Common Stock"] - cash.loc["Common Dividends Paid"]) / fcfe, years=years)
    }

    return result


def format_yy_growth_list(yy_growth: list):
    template = """
    <div style="width: 100%; display: flex;">
    <svg viewBox="0 0 {{items * 4}} 20" width="40" style='margin: auto; padding: 5px'>
    {% for l in lst %}
        <path d="M{{loop.index0 * 4}} {{scaling}} h 2 v {{l / max_val * -scaling}} h -2"></path>
    {% endfor %}
    </svg>
    </div
    """
    template = j2_env.from_string(template)
    yy_growth_adjusted = [x if not np.isnan(x) else 0 for x in yy_growth]
    return template.render(lst=yy_growth_adjusted, items=len(yy_growth), max_val=np.max(np.abs(yy_growth_adjusted)), scaling=10 if any([x < 0 for x in yy_growth]) else 20)


def read_tikr_html(filename):
    dfs = pd.read_html(filename)

    income = parse_table(dfs[0])
    balance = parse_table(dfs[1])
    cashflow = parse_table(dfs[2])

    # table = pd.DataFrame.from_dict({'TXRH': get_income_stats(income)}, 'index')
    income_table = pd.DataFrame(get_income_stats(income, years=10)).T
    balance_table = pd.DataFrame(get_balance_stats(income, balance, years=10)).T
    cash_table = pd.DataFrame(get_cash_stats(income, cashflow, years=10)).T

    return {
        'income': income_table,
        'balance': balance_table,
        'cashflow': cash_table,
    }

In [10]:
result = read_tikr_html("/home/mmeng/Dropbox/investing/AD/comparisons/data/Koninklijke Ahold Delhaize N.V. (AD).html")

In [13]:
result["income"].loc["net_income_margin"]

series         [0.02468189483366549, 0.024134985049124306, 0....
yy_growth      [0.0015690941392196844, 0.004240762971500445, ...
yy_growth_5                                            -0.026116
yy_growth_3                                             0.102826
yy_growth_1                                            -0.143419
growth_tot                                              0.015802
ltm                                                     0.025072
mean                                                    0.024972
std                                                     0.004341
mean_z2                                                 0.024972
std_z2                                                  0.004341
Name: net_income_margin, dtype: object

In [21]:
import os
files = list(filter(lambda x: x.endswith("html"), os.listdir("/home/mmeng/Dropbox/investing/AD/comparisons/data/")))
files

['Costco Wholesale Corporation (COST).html',
 'Koninklijke Ahold Delhaize N.V. (AD).html',
 'Carrefour SA (CA).html',
 'Walgreens Boots Alliance, Inc. (WBA).html',
 'Colruyt Group N.V. (COLR).html',
 'Tesco PLC (TSCO).html',
 'Walmart Inc. (WMT).html',
 'Jerónimo Martins, SGPS, S.A. (JMT).html',
 'Sprouts Farmers Market, Inc. (SFM).html',
 'Kesko Oyj (KESKOB).html',
 'Target Corporation (TGT).html',
 'The Kroger Co. (KR).html',
 'Metro Inc. (MRU).html']

In [30]:
financials = {}
for file in files:
    try:
        x = read_tikr_html("/home/mmeng/Dropbox/investing/AD/comparisons/data/" + file)
        financials[file[:-5]] = x
    except Exception:
        print("Skipping %s because exception was throws" % file)

Skipping Jerónimo Martins, SGPS, S.A. (JMT).html because exception was throws
Skipping Sprouts Farmers Market, Inc. (SFM).html because exception was throws


In [31]:
financials.keys()

dict_keys(['Costco Wholesale Corporation (COST)', 'Koninklijke Ahold Delhaize N.V. (AD)', 'Carrefour SA (CA)', 'Walgreens Boots Alliance, Inc. (WBA)', 'Colruyt Group N.V. (COLR)', 'Tesco PLC (TSCO)', 'Walmart Inc. (WMT)', 'Kesko Oyj (KESKOB)', 'Target Corporation (TGT)', 'The Kroger Co. (KR)', 'Metro Inc. (MRU)'])

In [57]:
net_income_margins = {}
interest_expense_margin = {}
dps_growth = {}
payout = {}

debt_to_earnings = {}

for f in financials.keys():
    net_income_margins[f] = financials[f]["income"].loc["net_income_margin"][["mean", "mean_z2"]]
    interest_expense_margin[f] = financials[f]["income"].loc["interest_expense_margin"][["mean", "mean_z2"]]
    dps_growth[f] = financials[f]["income"].loc["dividends"][["growth_tot"]]
    payout[f] = financials[f]["income"].loc["payout"][["mean", "mean_z2"]]

    debt_to_earnings[f] = financials[f]["balance"].loc["debt_to_earnings"][["mean", "mean_z2"]]


net_income_margin_df = pd.DataFrame(net_income_margins, index=["mean", "mean_z2"]).T.sort_values(by="mean_z2", ascending=False)
interest_expense_margin_df = pd.DataFrame(interest_expense_margin, index=["mean", "mean_z2"]).T.sort_values(by="mean_z2", ascending=True)
dps_growth_df = pd.DataFrame(dps_growth, index=["growth_tot"]).T.sort_values(by="growth_tot", ascending=False)
debt_to_earnings_df = pd.DataFrame(debt_to_earnings, index=["mean", "mean_z2"]).T.sort_values(by="mean_z2", ascending=True)
payout_df = pd.DataFrame(payout, index=["mean", "mean_z2"]).T.sort_values(by="mean_z2", ascending=True)

In [48]:
net_income_margin_df

,mean,mean_z2
Metro Inc. (MRU),0.051207,0.044394
Target Corporation (TGT),0.041017,0.038479
Colruyt Group N.V. (COLR),0.042511,0.037429
Kesko Oyj (KESKOB),0.028621,0.028621
Walmart Inc. (WMT),0.025338,0.026576
"Walgreens Boots Alliance, Inc. (WBA)",0.021564,0.025935
Koninklijke Ahold Delhaize N.V. (AD),0.024972,0.024972
Costco Wholesale Corporation (COST),0.023017,0.023017
The Kroger Co. (KR),0.016405,0.015522
Tesco PLC (TSCO),0.004947,0.015516


In [49]:
interest_expense_margin_df

,mean,mean_z2
Colruyt Group N.V. (COLR),0.009932,0.005941
Costco Wholesale Corporation (COST),0.028859,0.028859
Metro Inc. (MRU),0.098328,0.098328
Kesko Oyj (KESKOB),0.114971,0.100372
Walmart Inc. (WMT),0.098676,0.101532
Target Corporation (TGT),0.10476,0.110591
"Walgreens Boots Alliance, Inc. (WBA)",0.150515,0.150515
Carrefour SA (CA),0.169636,0.160824
Koninklijke Ahold Delhaize N.V. (AD),0.164275,0.164275
The Kroger Co. (KR),0.167238,0.167238


In [50]:
dps_growth_df

,growth_tot
The Kroger Co. (KR),2.4375
Kesko Oyj (KESKOB),2.085714
Metro Inc. (MRU),2.025
Costco Wholesale Corporation (COST),1.977444
Target Corporation (TGT),1.746835
Koninklijke Ahold Delhaize N.V. (AD),1.0
"Walgreens Boots Alliance, Inc. (WBA)",0.5
Walmart Inc. (WMT),0.207447
Carrefour SA (CA),-0.096774
Colruyt Group N.V. (COLR),-0.2


In [54]:
debt_to_earnings_df

,mean,mean_z2
Costco Wholesale Corporation (COST),-0.390146,-0.390146
Colruyt Group N.V. (COLR),0.147198,-0.319239
Kesko Oyj (KESKOB),3.323844,2.104899
Metro Inc. (MRU),3.405612,3.405612
Walmart Inc. (WMT),3.825523,3.447371
Target Corporation (TGT),2.832132,3.759538
Koninklijke Ahold Delhaize N.V. (AD),4.381171,4.381171
The Kroger Co. (KR),8.069848,7.623186
"Walgreens Boots Alliance, Inc. (WBA)",15.491272,8.226364
Carrefour SA (CA),5.930509,11.046797


In [58]:
payout_df

,mean,mean_z2
The Kroger Co. (KR),0.259393,0.242904
Metro Inc. (MRU),0.254813,0.270235
Costco Wholesale Corporation (COST),0.287152,0.287152
Carrefour SA (CA),0.210759,0.297283
Walmart Inc. (WMT),0.49166,0.448987
Target Corporation (TGT),0.340717,0.449128
Colruyt Group N.V. (COLR),0.433967,0.467286
Tesco PLC (TSCO),0.474627,0.474627
Koninklijke Ahold Delhaize N.V. (AD),0.507528,0.538458
"Walgreens Boots Alliance, Inc. (WBA)",0.857142,0.587647
